In [2]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Define directories where the train, test, and validation images are stored
train_dir = '/kaggle/input/bttai-nybg-2024/BTTAIxNYBG-train/BTTAIxNYBG-train'
test_dir = '/kaggle/input/bttai-nybg-2024/BTTAIxNYBG-test/BTTAIxNYBG-test'
validation_dir = '/kaggle/input/bttai-nybg-2024/BTTAIxNYBG-validation/BTTAIxNYBG-validation'

# Load dataframes containing metadata such as image filenames and class labels
train_df = pd.read_csv('/kaggle/input/bttai-nybg-2024/BTTAIxNYBG-train.csv')
test_df = pd.read_csv('/kaggle/input/bttai-nybg-2024/BTTAIxNYBG-test.csv')
validate_df = pd.read_csv('/kaggle/input/bttai-nybg-2024/BTTAIxNYBG-validation.csv')

# Data augmentation configuration for the training set to improve model generalization
train_datagen = ImageDataGenerator(
    rotation_range=40,  # Degrees of random rotations
    width_shift_range=0.2,  # Fraction of total width, for horizontal shift
    height_shift_range=0.2,  # Fraction of total height, for vertical shift
    shear_range=0.2,  # Shear Intensity (Shear angle in counter-clockwise direction)
    zoom_range=[0.8, 1.2],  # Range for random zoom. Now allows for zoom in and out
    horizontal_flip=True,  # Randomly flip inputs horizontally
    fill_mode='nearest',  # Strategy to fill in newly created pixels
    brightness_range=[0.5, 1.5],  # Randomly alter the brightness of images
    channel_shift_range=50.0,  # Range for random channel shifts
    rescale=1./255  # Rescaling factor for normalizing pixel values
)

# Data generators for validation and test sets only rescale the images
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Converts dataframe to a data generator (suitable for model training)
def df_to_dataset(dataframe, datagen, directory, batch_size=32):
    return datagen.flow_from_dataframe(
        dataframe=dataframe,
        directory=directory,
        x_col='imageFile',  # Column in dataframe that contains the filenames
        y_col='classLabel',  # Column in dataframe that contains the class/label
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical'  # Multiclass classification
    )

# Create datasets for training and validation
train_dataset = df_to_dataset(train_df, train_datagen, train_dir)
validation_dataset = df_to_dataset(validate_df, validation_datagen, validation_dir)

# Load MobileNetV2 pretrained on ImageNet and freeze the first 'fine_tune_at' layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
fine_tune_at = 100
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Assemble the full model including new top layers
model = Sequential([
    base_model,
    # Convert features to vectors
    tf.keras.layers.GlobalAveragePooling2D(),
    # Add a dense layer for classification
    Dense(1024, activation='relu'),
    # Final layer with softmax activation for multi-class classification
    Dense(10, activation='softmax')
])

# Set up learning rate schedule and compile model
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=10000,
    decay_rate=0.9
)
model.compile(optimizer=Adam(learning_rate=lr_schedule), loss='categorical_crossentropy', metrics=['accuracy'])
# Train the model on a subset of the training data for quick testing
train_subset = train_df.sample(frac=0.7, random_state=42)
train_subset_dataset = df_to_dataset(train_subset, train_datagen, train_dir)

# Callbacks for early stopping and model checkpointing
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True),
    ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)
]

# Train the model
history = model.fit(
    train_subset_dataset,  # Use the subset of data
    validation_data=validation_dataset,
    epochs=3,  # Initially train for fewer epochs for debugging
    callbacks=callbacks
)

# Continue training on the entire dataset
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=4, 
    callbacks=callbacks
)

# Evaluate the model on the validation set
validation_loss, validation_accuracy = model.evaluate(validation_dataset)
print(f'Validation Loss: {validation_loss}')
print(f'Validation Accuracy: {validation_accuracy}')

# Generate predictions on the test set
test_dataset = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=test_dir,
    x_col='imageFile',  # Make sure column name matches test_df column name for filenames
    target_size=(224, 224),
    batch_size=32,
    class_mode=None,  # No labels
    shuffle=False
)
predictions = model.predict(test_dataset)
predicted_class_indices = np.argmax(predictions, axis=1)

# Save predictions to a CSV file for submission
submission_df = pd.DataFrame({'uniqueID': test_df['uniqueID'], 'classID': predicted_class_indices})
submission_df.to_csv('/kaggle/working/submission.csv', index=False)


Found 81946 validated image filenames belonging to 10 classes.
Found 10244 validated image filenames belonging to 10 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Found 57362 validated image filenames belonging to 10 classes.
Epoch 1/3


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


   1/1793 ━━━━━━━━━━━━━━━━━━━━ 16:37:44 33s/step - accuracy: 0.0938 - loss: 2.5207

I0000 00:00:1717981261.157718      82 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1793/1793 ━━━━━━━━━━━━━━━━━━━━ 1671s 914ms/step - accuracy: 0.8422 - loss: 0.4764 - val_accuracy: 0.8627 - val_loss: 0.5227
Epoch 2/3
1793/1793 ━━━━━━━━━━━━━━━━━━━━ 1272s 707ms/step - accuracy: 0.9306 - loss: 0.1963 - val_accuracy: 0.9379 - val_loss: 0.1863
Epoch 3/3
1793/1793 ━━━━━━━━━━━━━━━━━━━━ 1265s 703ms/step - accuracy: 0.9468 - loss: 0.1560 - val_accuracy: 0.9489 - val_loss: 0.1636
Restoring model weights from the end of the best epoch: 3.
Epoch 1/4
2561/2561 ━━━━━━━━━━━━━━━━━━━━ 2149s 837ms/step - accuracy: 0.9491 - loss: 0.1421 - val_accuracy: 0.9398 - val_loss: 0.1980
Epoch 2/4
2561/2561 ━━━━━━━━━━━━━━━━━━━━ 2186s 851ms/step - accuracy: 0.9549 - loss: 0.1260 - val_accuracy: 0.9619 - val_loss: 0.1192
Epoch 3/4
2561/2561 ━━━━━━━━━━━━━━━━━━━━ 1962s 764ms/step - accuracy: 0.9598 - loss: 0.1144 - val_accuracy: 0.9625 - val_loss: 0.1113
Epoch 4/4
2561/2561 ━━━━━━━━━━━━━━━━━━━━ 1902s 741ms/step - accuracy: 0.9627 - loss: 0.1044 - val_accuracy: 0.9491 - val_loss: 0.1594
Restoring mod